In [ ]:
!pip install imutils

In [ ]:
import joblib

# Save the model
def save_model(model, path):
    #joblib.dump(svm_clf_dct, 'svm_dct_model.pkl')
    joblib.dump(model, path)
    
def load_model(path):
    # Load the model
    #svm_clf_dct_local = joblib.load('/kaggle/input/svm-dct-model/svm_dct_model.pkl')
    return(joblib.load(path))


# CLASSIC METHODS

import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tqdm import tqdm  # for progress bar
import random 

import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from imutils import paths
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score 
from skimage.filters import gabor
from scipy.spatial.qhull import QhullError
from scipy import spatial
spatial.QhullError = QhullError
from skimage.feature import local_binary_pattern
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
def colortogray(im):
    image = cv2.imread(im)
    imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return imgray

def resizeImage(image, size):
    return cv2.resize(image, (size,size))

In [ ]:
def extract_features_labels(directory):
    features = []
    labels = []
    for emotion_folder in os.listdir(directory):
        emotion_folder_path = os.path.join(directory, emotion_folder)
        
        # Check if it's a directory
        if os.path.isdir(emotion_folder_path):
            print(f"Processing {emotion_folder} images...")
        for image_file in tqdm(os.listdir(emotion_folder_path)):
                image_path = os.path.join(emotion_folder_path, image_file)
                image = colortogray(image_path)
                image = resizeImage(image, 64)

                fd1 = fd1 =  hog(image, orientations=7, pixels_per_cell=(8, 8),cells_per_block=(4, 4),block_norm= 'L2-Hys' ,transform_sqrt = False)
                label = emotion_folder
                labels.append(label)
                features.append(fd1)
    features = np.array(features)
    labels = np.array(labels)
    return features,labels

In [ ]:
from scipy.fftpack import dct, idct

# implement 2D DCT
def dct2_method(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')

# implement 2D IDCT
def idct2_method(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho')

def lbp_method(image, P, R):
    lbp = local_binary_pattern(image, P, R, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist

def gabor_filters_method(image, frequencies):
    responses = []
    for frequency in frequencies:
        real, imag = gabor(image, frequency=frequency)
        magnitude = np.sqrt(real**2 + imag**2)
        responses.append(magnitude.flatten())  # Flatten each response
    combined_response = np.concatenate(responses, axis=0)  # Concatenate flattened responses
    return combined_response


In [ ]:
# DISPLAY METHODS

import math
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

def view_random_images(directory, target_class, num_of_samples=8):
  target_folder = os.path.join(directory, target_class)

  random_images = random.sample(os.listdir(target_folder), num_of_samples)
  print(random_images)
  # Create a subplot to display multiple images
  plt.figure(figsize=(12, 6))
  for i, image in enumerate(random_images):
    plt.subplot(math.ceil(num_of_samples / 4), 4, i + 1)  # 2 rows, 4 columns
    img = cv2.imread(os.path.join(target_folder, image))
    
    plt.imshow(img)
    plt.title(f"{target_class} - {i + 1}")
    plt.axis("off")

  plt.show()

def get_sample_of_images(directory, num_of_samples = 8):    
    for emotion in emotions:
        view_random_images(directory, emotion, num_of_samples)

def get_sample_of_feature_extractions(directory, num_of_samples = 8, method_name = 'dct'):
    for emotion_target in emotions:
        target_folder = os.path.join(directory, emotion_target)
        random_images = random.sample(os.listdir(target_folder), num_of_samples)
        print(emotion_target)
        for i, image in enumerate(random_images):
            img = cv2.imread(os.path.join(target_folder, image))
            print(image)
            print(use_method_on_image(img, method_name))
    
def use_method_on_image(image, method_name = 'dct'):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_gray = resizeImage(img_gray, 64)
    
    if method_name == 'dct':
        return dct2_method(img_gray)
    elif method_name == 'idct':
        return idct2_method(img_gray)
    elif method_name == 'lbp':
        return lbp_method(img_gray, P=8, R=1)
    elif method_name == 'gabor':
        return gabor_filters_method(img_gray, frequencies=[0.6, 1.0])

In [ ]:
images_directory = '/kaggle/input/face-expression-recognition-dataset/images/train'
#get_sample_of_images(images_directory)

get_sample_of_feature_extractions(images_directory, 4, 'gabor')


In [ ]:
def extract_features_labels_by_method(directory, method, total_input_count = -1):
    features = []
    labels = []
    for emotion_folder in os.listdir(directory):
        emotion_folder_path = os.path.join(directory, emotion_folder)
        
        if os.path.isdir(emotion_folder_path):
            print(f"Processing {emotion_folder} images...")
            count = 0
            for image_file in tqdm(os.listdir(emotion_folder_path)):
                if total_input_count > 0 and count >= total_input_count / 3:
                    continue
                count += 1
                image_path = os.path.join(emotion_folder_path, image_file)
                img = cv2.imread(image_path)
                features.append(use_method_on_image(img, method))
                label = emotion_folder
                labels.append(label)
    return features, labels
def extract_features_labels_all_methods(directory, method_list, total_input_count = -1):
    all_features = []
    for method_name in method_list:
        extract_features_labels_by_method(directory, method_name, total_input_count)

In [ ]:
images_directory = '/kaggle/input/face-expression-recognition-dataset/images/train'
features_dct, labels_dct = extract_features_labels_by_method(images_directory, 'dct')

In [ ]:
def flatten_images(images):
    flattened_images = []
    for img in images:
        # Flatten the image and append it to the flattened_images list
        flattened_images.append(img.flatten())
    return flattened_images

In [ ]:
# DCT MODEL
R =random.randint(1,88)
X_train_dct, X_test_dct, y_train_dct, y_test_dct = train_test_split(features_dct, labels_dct, test_size= 0.3,random_state=R)


X_train_dct_array = np.array(X_train_dct)
nsamples, nx, ny = X_train_dct_array.shape
d2_train_dataset = X_train_dct_array.reshape((nsamples,nx*ny))
print(nsamples, nx, ny)

svm_clf_dct = SVC(kernel='rbf', gamma='scale', C= 10)
svm_clf_dct.fit(d2_train_dataset, y_train_dct)

In [ ]:
def predict_and_display(model, image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img_processed = preprocess_image(img)  # Replace with your actual preprocessing steps

    # Predict the expression
    prediction = model.predict([img_processed])[0]

    # Display the image and prediction
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert color from BGR to RGB
    plt.title(f"Predicted Expression: {prediction}")
    plt.axis("off")
    plt.show()

def preprocess_image(image):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    processed_image = resizeImage(img_gray, 64)
    processed_image = processed_image.flatten()
    return processed_image


In [ ]:
predict_and_display(svm_clf_dct_local, '/kaggle/input/face-expression-recognition-dataset/images/train/angry/0.jpg')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Veri seti yolu
train_directory = '/kaggle/input/face-expression-recognition-dataset/images/train'
test_directory = '/kaggle/input/face-expression-recognition-dataset/images/validation'

# Veri artırma ve ön işleme
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Eğitim ve test veri setlerini yükleme
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical')

# CNN modeli oluşturma
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(7, activation='softmax')  # Sınıf sayısına göre ayarlayın
])

# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(train_generator, epochs=10, validation_data=test_generator)
